# Execute all versions of lid driven cavity problem

In [10]:
from pathlib import Path
from xml.etree import ElementTree
path = "bin"
programs = {
    "Lid driven cavity": "lidDriven",
    "Lid driven cavity GPU": "lidDrivenGPU"
}
hs = [h/1000.0 for h in range(20, 6, -2)]

[0.02, 0.018, 0.016, 0.014, 0.012, 0.01, 0.008]

Create parameter files

In [18]:
parameter_path = Path("..", "params")
default_param = parameter_path / "chanelFlow.xml"
for name, program in programs.items():
    params = ElementTree.parse(default_param)
    root = params.getroot()
    for h in hs:
        num_xml = root.find("./num")
        num_xml.set('h', str(h))
        output = root.find("./output")
        output.set("use_xml_name", str(1))
        output.set("hdf5_name", f"benchmark/{program}_{int(h*1000):0>2}_results.h5")
        params.write(parameter_path / f"benchmark/chanelFlow_{program}_{int(h*1000):0>2}.xml")

Run program for every parameter file

In [20]:
for name, program in programs.items():
    for h in hs:
        param_path = Path("..", "params", "benchmark", f"chanelFlow_{program}_{int(h*1000):0>2}.xml")
        program_path = Path("..", "bin", program)
        output_path = Path("..", "benchmark", f"{program}_{int(h*1000):0>2}_results.h5")
        if Path.exists(output_path):
            continue
        print("Executing", name, "for h =", h)
        !../bin/{program} {param_path} 


Executing Lid driven cavity for h = 0.02
h:0.02, N: 2277, dt:0.1
10 - t:1 max u_y:0.0234295 @ x:0.0908523  (max div:6.06618)
20 - t:2 max u_y:0.0252587 @ x:0.128638  (max div:3.6495)
30 - t:3 max u_y:0.0311025 @ x:0.204768  (max div:1.77669)
40 - t:4 max u_y:0.0379845 @ x:0.204768  (max div:0.885512)
50 - t:5 max u_y:0.0461333 @ x:0.377819  (max div:0.672332)
60 - t:6 max u_y:0.0723569 @ x:0.53903  (max div:0.41808)
70 - t:7 max u_y:0.12304 @ x:0.53903  (max div:0.322734)
80 - t:8 max u_y:0.154041 @ x:0.479766  (max div:0.305433)
90 - t:9 max u_y:0.183781 @ x:0.455599  (max div:0.241322)
100 - t:10 max u_y:0.196802 @ x:0.455599  (max div:0.201558)
110 - t:11 max u_y:0.210226 @ x:0.377819  (max div:0.163063)
120 - t:12 max u_y:0.227005 @ x:0.377819  (max div:0.121454)
130 - t:13 max u_y:0.233739 @ x:0.377819  (max div:0.0984679)
140 - t:14 max u_y:0.240554 @ x:0.341837  (max div:0.0999758)
150 - t:15 max u_y:0.247877 @ x:0.321885  (max div:0.115578)
160 - t:16 max u_y:0.25553 @ x:0.3019

In [3]:
import os
os.getcwd()

'/home/ggolob/lid-driven-cavity-gpu/benchmark'